In [10]:
import myComponents as tc
import topic as tp
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import os

In [11]:
# network parameter
mode = 'SF'

num_broker = 1000
num_sub = 8000
num_pub = 2000


broker_r = 300
broker_rates = [broker_r]*num_broker
topic_dist = [5,5,5,5,5]
monitor_rate = 1
runtime = 100
connection_style = None

# sub parameter
avg_sub_size = 10.0
sub_r = 3

sub_rates = [sub_r]*num_sub
sub_num_topic = [100]*num_sub
sub_diameter = [3]*num_sub

# pub parameter
avg_pub_size = 20.0
pub_r = 3

pub_rates = [pub_r]*num_pub
pub_num_topic = [100]*num_pub
pub_diameter = [3]*num_pub


wildcard_rate=1
plus_rate=0
hash_rate=0


seed = 1


In [12]:
dir_name = str(num_broker)+'_'+str(num_sub)+'_'+str(num_pub)+'_'+str(sub_r)+'_'+str(pub_r)+'_'+str(wildcard_rate*plus_rate)+'_'+str(wildcard_rate*hash_rate)
os.makedirs('./data/'+dir_name, exist_ok = True)

sub_waits = open('./data/'+dir_name+'/'+mode+'_'+'sub_waits.csv', 'w')
sub_pkt = open('./data/'+dir_name+'/'+mode+'_'+'sub_pkt.csv', 'w')
broker_queue = open('./data/'+dir_name+'/'+mode+'_'+'broker_queue.csv', 'w')
broker_output = open('./data/'+dir_name+'/'+mode+'_'+'broker_output.csv', 'w')

tc.SwitchPort.mode = mode
total_topic = tp.TopicTree(wildcard_rate, plus_rate, hash_rate)
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size, sub_waits, sub_pkt, broker_queue, broker_output, qlimit=None)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
print(tc.SwitchPort.mode)
    
net.env.run(runtime)
    
sub_waits.close()
sub_pkt.close()
broker_queue.close()
broker_output.close()

SF
current time:  5
current time:  10
current time:  15
current time:  20
current time:  25
current time:  30
current time:  35
current time:  40
current time:  45
current time:  50
current time:  55
current time:  60
current time:  65
current time:  70
current time:  75
current time:  80
current time:  85
current time:  90
current time:  95


In [ ]:
wildcard_rate=1
plus_rate=0.1
hash_rate=0.01
dir_name = str(num_broker)+'_'+str(num_sub)+'_'+str(num_pub)+'_'+str(sub_r)+'_'+str(pub_r)+'_'+str(wildcard_rate*plus_rate)+'_'+str(wildcard_rate*hash_rate)
os.makedirs('./data/'+dir_name, exist_ok = True)

sub_waits = open('./data/'+dir_name+'/'+mode+'_'+'sub_waits.csv', 'w')
sub_pkt = open('./data/'+dir_name+'/'+mode+'_'+'sub_pkt.csv', 'w')
broker_queue = open('./data/'+dir_name+'/'+mode+'_'+'broker_queue.csv', 'w')
broker_output = open('./data/'+dir_name+'/'+mode+'_'+'broker_output.csv', 'w')

tc.SwitchPort.mode = mode
total_topic = tp.TopicTree(wildcard_rate, plus_rate, hash_rate)
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size, sub_waits, sub_pkt, broker_queue, broker_output, qlimit=None)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
print(tc.SwitchPort.mode)
    
net.env.run(runtime)
    
sub_waits.close()
sub_pkt.close()
broker_queue.close()
broker_output.close()

In [ ]:
wildcard_rate=1
plus_rate=0.3
hash_rate=0.03

dir_name = str(num_broker)+'_'+str(num_sub)+'_'+str(num_pub)+'_'+str(sub_r)+'_'+str(pub_r)+'_'+str(wildcard_rate*plus_rate)+'_'+str(wildcard_rate*hash_rate)
os.makedirs('./data/'+dir_name, exist_ok = True)

sub_waits = open('./data/'+dir_name+'/'+mode+'_'+'sub_waits.csv', 'w')
sub_pkt = open('./data/'+dir_name+'/'+mode+'_'+'sub_pkt.csv', 'w')
broker_queue = open('./data/'+dir_name+'/'+mode+'_'+'broker_queue.csv', 'w')
broker_output = open('./data/'+dir_name+'/'+mode+'_'+'broker_output.csv', 'w')

tc.SwitchPort.mode = mode
total_topic = tp.TopicTree(wildcard_rate, plus_rate, hash_rate)
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size, sub_waits, sub_pkt, broker_queue, broker_output, qlimit=None)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
print(tc.SwitchPort.mode)
    
net.env.run(runtime)
    
sub_waits.close()
sub_pkt.close()
broker_queue.close()
broker_output.close()

In [ ]:
wildcard_rate=1
plus_rate=0.5
hash_rate=0.05
dir_name = str(num_broker)+'_'+str(num_sub)+'_'+str(num_pub)+'_'+str(sub_r)+'_'+str(pub_r)+'_'+str(wildcard_rate*plus_rate)+'_'+str(wildcard_rate*hash_rate)
os.makedirs('./data/'+dir_name, exist_ok = True)

sub_waits = open('./data/'+dir_name+'/'+mode+'_'+'sub_waits.csv', 'w')
sub_pkt = open('./data/'+dir_name+'/'+mode+'_'+'sub_pkt.csv', 'w')
broker_queue = open('./data/'+dir_name+'/'+mode+'_'+'broker_queue.csv', 'w')
broker_output = open('./data/'+dir_name+'/'+mode+'_'+'broker_output.csv', 'w')

tc.SwitchPort.mode = mode
total_topic = tp.TopicTree(wildcard_rate, plus_rate, hash_rate)
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size, sub_waits, sub_pkt, broker_queue, broker_output, qlimit=None)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
print(tc.SwitchPort.mode)
    
net.env.run(runtime)
    
sub_waits.close()
sub_pkt.close()
broker_queue.close()
broker_output.close()

tc.SwitchPort.mode = 'SF'
total_topic = tp.TopicTree()
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
net.env.run(runtime)
print(tc.SwitchPort.mode)
net.visualize()
plt.figure(figsize=(10, 3), dpi=80)
#monitor.queue_size
#monitor.queue_length
for i in range(len(net.broker_monitor_list)):
    monitor = net.broker_monitor_list[i]
    plt.plot(monitor.queue_length, label=monitor.port.sp_id)
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.title("queue length")
window = 6
plt.figure(figsize=(10, 3), dpi=80)
for i in range(len(net.broker_monitor_list)):
    monitor = net.broker_monitor_list[i]
    a = np.array(monitor.bytes_sent)
    b = np.array([0]*window+monitor.bytes_sent[0:-window])
    plt.plot((a-b)[window-1:]/window/monitor_rate, label=monitor.port.sp_id)
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.title("output rate in byte/s, window="+str(window))
import warnings
warnings.filterwarnings('ignore')
window = 6
plt.figure(figsize=(10, 9), dpi=80)

for i in range(len(net.sub_monitor_list)):
    monitor = net.sub_monitor_list[i]
    a = np.array(monitor.packets_rec)
    b = np.array([0]*window+monitor.packets_rec[0:-window])
    avg_packet_rec_per_time = (a-b)[window-1:]/window/monitor_rate
    
    a = np.array(monitor.tot_waits)
    b = np.array([0]*window+monitor.tot_waits[0:-window])
    avg_waits_per_time = (a-b)[window-1:]/window/monitor_rate
    

    plt.subplot(311)
    plt.plot(avg_waits_per_time, label=monitor.client.client_id)
    plt.subplot(312)
    plt.plot(avg_packet_rec_per_time, label=monitor.client.client_id)
    plt.subplot(313)
    plt.plot(avg_waits_per_time/avg_packet_rec_per_time, label=monitor.client.client_id)

plt.subplot(311)
plt.title("avg_waits s/s, window="+str(window))
plt.subplot(312)
plt.title("avg_pkt_recieved /s, window="+str(window))
plt.subplot(313)
plt.title("avg_waits/pkt, window="+str(window))
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,wspace=0.35)